<a href="https://colab.research.google.com/github/ahmedlabib02/Nlp-project/blob/main/Nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 1

In [42]:
import os 
import json
import re
import pandas as pd

### Data preparation

#### Unzipping the "B Hodoo2" channel folder

In [25]:
import zipfile
zip_path = "./B Hodoo2-20250226T182935Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


#### Unzipping the "Kefaya ba2a" channel folder

In [26]:
import zipfile
zip_path = "./Kefaya Ba2a-20250226T183527Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### Building dict for annotations and text

In [28]:
def standardize(title):
    
    title = title.replace("_", " ")
   
    title = title.lower()
    
    title = re.sub(r'[^\w\s]', '', title)
    
    # title = re.sub(r'\bبودكاست بهدوء مع كريم\b', '', title)
    # title = re.sub(r'\bجلسة\b', '', title)
    
    title = re.sub(r'\s+', ' ', title).strip()
    return title

In [30]:
def get_category_for_filename(target_filename, annotations_path):
    """
    Load annotations from annotations_path and try to match the
    canonicalized target_filename (without file extension) to the canonicalized
    title from each annotation. If a match is found, return the annotation's category.
    """
    with open(annotations_path, 'r', encoding='utf-8') as f_json:
        annotations = json.load(f_json)
    
    base_target = os.path.splitext(target_filename)[0]
    target_can = standardize(base_target)
    for entry in annotations:
        annotation_title = entry.get('title')
        if annotation_title:
            
            annotation_can = standardize(annotation_title)
            if annotation_can == target_can:
                # print("Matched annotation:", entry.get('title'))
                return entry.get('category')
    return None

In [31]:
def add_transcripts_from_folder(transcripts_dict, folder_path, channel):
    """
    For each file in the 'raw' subfolder of folder_path, read the transcript,
    obtain its category by matching the filename to the annotation (using canonicalization),
    and store the (channel, category, transcript) tuple in transcripts_dict.
    """
    annotations_json = os.path.join(folder_path, 'annotations.json')
    raw_folder = os.path.join(folder_path, 'raw')
    for filename in os.listdir(raw_folder):
        full_path = os.path.join(raw_folder, filename)
        if os.path.isfile(full_path):
            with open(full_path, 'r', encoding='utf-8') as file:
                transcript = file.read()
                category = get_category_for_filename(filename, annotations_json)
            base_filename, _ = os.path.splitext(filename)
            transcripts_dict[base_filename] = (channel, category, transcript)
    return transcripts_dict

In [32]:
transcripts_dict = {}
first_folder_path = './B Hodoo2/'

transcripts_dict = add_transcripts_from_folder(transcripts_dict, first_folder_path,  'B Hodoo2')

print(dict(list(transcripts_dict.items())[:2]))


{'أكبر كذبة في التاريخ _ وهم الانتاجية! - _ بودكاست بهدوء مع كريم _ جلسة 19': ('B Hodoo2', 'Education', 'النهارده الفيديو بتاعنا على\nالانتاجيه لان النهارده الفيديو رقم 500 ل\nهي على قناه اليوتيوب وبحسبه صغيره صديقي\nالمشاهد فيديو كل اسبوع يعني اربع\nفيديوهات في الشهر اربع فيديوهات في الشهر\nيعني 12 يعني 48 فيديو في السنه ولو\nضربناهم في الثمان سنين هيبقوا اقل من 500\nفاذا انا شخصيا بقي اكت من ممان سنين\nمنتجه على اليوتيوب فيحق ل ان انا اكلمك\nعن الانتاجيه بس الحقيقه ان ما يحق ليش\nخالص اكلمك على الانتاجيه لو ضربت المثل\nده لان المثل ده هو الارقام اللي اتقالت\nفيه كلها صحيحه بدون اي كذب بس الحقيقه لو\nانت متابع جيد واعمل سبسكرايب ومفعل الجرس\nهتعرف ان كريم اسماعيل مش شخصيه ملتزمه في\nاليوتيوب وما بينزلش فيديو كل اسبوع لمده\nممان سنين واللي عمل السبايك بتاعه\nالارقام ديت ان انا جت عل فتره كنت بنزل\nفيديو كل يوم لكن غير كده انا عمري ما\nاستمريت اكتر من لاث شهور على اليوتيوب\nبنزل فيديو كل اسبوع لان اليوتيوب اصلا مش\nمهنتي الاساسيه ومش شغلتي الاساسيه و\nبالتالي انا مش محتاج اصلا التزم بفي

In [33]:
second_folder_path = './Kefaya Ba2a/raw'



for filename in os.listdir(first_folder_path):
    full_path = os.path.join(first_folder_path, filename)
    if os.path.isfile(full_path):
        with open(full_path, 'r', encoding='utf-8') as file:
            transcripts_dict[filename] = file.read()

# filename -> (channel, content, category)
transcripts_dict

{'أكبر كذبة في التاريخ _ وهم الانتاجية! - _ بودكاست بهدوء مع كريم _ جلسة 19': ('B Hodoo2',
  'Education',
  'النهارده الفيديو بتاعنا على\nالانتاجيه لان النهارده الفيديو رقم 500 ل\nهي على قناه اليوتيوب وبحسبه صغيره صديقي\nالمشاهد فيديو كل اسبوع يعني اربع\nفيديوهات في الشهر اربع فيديوهات في الشهر\nيعني 12 يعني 48 فيديو في السنه ولو\nضربناهم في الثمان سنين هيبقوا اقل من 500\nفاذا انا شخصيا بقي اكت من ممان سنين\nمنتجه على اليوتيوب فيحق ل ان انا اكلمك\nعن الانتاجيه بس الحقيقه ان ما يحق ليش\nخالص اكلمك على الانتاجيه لو ضربت المثل\nده لان المثل ده هو الارقام اللي اتقالت\nفيه كلها صحيحه بدون اي كذب بس الحقيقه لو\nانت متابع جيد واعمل سبسكرايب ومفعل الجرس\nهتعرف ان كريم اسماعيل مش شخصيه ملتزمه في\nاليوتيوب وما بينزلش فيديو كل اسبوع لمده\nممان سنين واللي عمل السبايك بتاعه\nالارقام ديت ان انا جت عل فتره كنت بنزل\nفيديو كل يوم لكن غير كده انا عمري ما\nاستمريت اكتر من لاث شهور على اليوتيوب\nبنزل فيديو كل اسبوع لان اليوتيوب اصلا مش\nمهنتي الاساسيه ومش شغلتي الاساسيه و\nبالتالي انا مش محتاج اصلا التزم

### Creating a dataframe

In [43]:


data = [(title, info[0], info[1], info[2]) for title, info in transcripts_dict.items()]

df = pd.DataFrame(data, columns=['title', 'channel', 'category', 'transcript'])




### Cleaning the data

In [49]:
def clean_text(text):
  
    text = text.replace("_", " ")
    
    # Remove punctuation (keep Arabic letters and digits)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [50]:
file_path = "B Hodoo2\\raw\\٧ دروس من تجربتي في الحج ٢٠٢٣ _ بودكاست بهدوء مع كريم _ جلسة 12.txt" 
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

print(clean_text(text))



تقريبا مفيش حد فينا وهو صغير ما شفش في التلفزيون ماسك الحج وشاف الحجاج والناس كتير والدنيا زحمه وسمعنا من اهالينا على المشقه بتاعه التعب وفي المدرسه خدنا النوم الاركان الخمسه بس بصراحه انا كل ده ما كنتش يعني مستوعب الموضوع قوي وحقيقي مش مستوعبه حد لحد ما بدات في نفسه يعني واعتقد انه قرار ان انا اروح الحج دايما الناس بتقول ان هو قرار بس الحقيقه ان هو رزق من الله سبحانه وتعالى ولكنه كان حتى نابع من فكره ان الانسان يجب عليه ان يكمل دينه ان هو اذا توفر معه القيمه بتاعه الحج يجب ان يحج يعني كان امر كله مرتبط برضو بالاركان بتاعه الاسلام وجزء ايماني ما كانش مرتبط بالحج نفسه لاني ما اعرفوش يعني لحد ما سبحان الله ربنا اذن رحت يعني احنا سافرنا عشره ايام طبعا الحج عشره ايام كان منهم ايام في المدينه بعد كده في المناسك موسيقى ايام مغيره سبحان الله يعني فحاسس انه جزء من الواجب انا عارف في ناس تقعد تقول لك هذا سمعه وهذه لا لا احنا سبحان الله عندنا جهر بالمعاصي غير مسبوق في الزمن ده ونيجي احنا بس لما نيجي نعمل طاعه الناس كلها عايزه تحطنا في ملايات وما حدش يتكلم فنسال الله سبحانه وتعالى ان يجعل اعمالنا

In [ ]:
import os
import json

# Set directory where JSON files are stored
directory = "./annotations" 
output_file = "combined_annotations.json"

# Retreiving all json files in annotations directory
json_files = [f for f in os.listdir(directory) if f.endswith(".json")]


combined_data = []

for file in json_files:
    file_path = os.path.join(directory, file)
    
    with open(file_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if isinstance(data, dict):  
                combined_data.append(data) 
            elif isinstance(data, list):
                combined_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Error reading {file}: {e}")

with open(output_file, "w", encoding="utf-8") as outfile:
    json.dump(combined_data, outfile, indent=4, ensure_ascii=False)

print(f"Merged {len(json_files)} files into {output_file}")


Merged 21 files into combined_annotations.json
